# Ignition 8.1.36 Historian (Pacific Woodtech)

## History MainView

In [ ]:
#pendingPens 
#Change Script
def valueChanged(self, previousValue, currentValue, origin, missedEvents):
    if currentValue.value:
        self.getChild("root").getChild("CoordinateContainer").getChild("PowerChart").props.pens = currentValue.value

| Line | Code                                                                                                           | What it does                                                                                                                                                          |
| ---: | -------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def valueChanged(self, previousValue, currentValue, origin, missedEvents):`                                   | Defines the Perspective property change event handler. Runs when the bound property value changes. `self` is the component the script is attached to.                 |
|    2 | `if currentValue.value:`                                                                                       | Checks whether the new value is truthy. Prevents writing `None`, empty string, empty list, or `False` into the PowerChart pens.                                       |
|    3 | `self.getChild("root").getChild("CoordinateContainer").getChild("PowerChart").props.pens = currentValue.value` | Navigates down the view component tree to the `PowerChart` component, then assigns its `props.pens` property to the new incoming value so the chart updates its pens. |


| Parameter       | What it is in Ignition                                         | How it is used here                                      |
| --------------- | -------------------------------------------------------------- | -------------------------------------------------------- |
| `self`          | The component where this script lives                          | Used to navigate to other components via `getChild(...)` |
| `previousValue` | The prior value of the property that changed                   | Not used in this snippet                                 |
| `currentValue`  | The new value of the property that changed (a QualifiedValue)  | Uses `currentValue.value` as the pens payload            |
| `origin`        | Indicates what caused the change (binding, user, script, etc.) | Not used here                                            |
| `missedEvents`  | Number of missed change events if updates were coalesced       | Not used here                                            |


| Expression           | Meaning                                                  | Notes                                                        |
| -------------------- | -------------------------------------------------------- | ------------------------------------------------------------ |
| `currentValue.value` | The raw new value (unqualified)                          | Must match the structure expected by `PowerChart.props.pens` |
| `getChild("name")`   | Retrieves a child component by its name in the view tree | Chained calls walk down nested containers                    |
| `.props.pens`        | The PowerChart pens configuration array                  | Assigning this updates which pens the chart displays         |


### PowerChart Container

In [4]:
#PowerChart - Message Handler
def onMessageReceived(self, payload):
    tags = payload.get('tags', [])
    pens = []
    for tag in tags:
        tagName = tag.split('/')[-1]
        dataSource = 'histprov:BiP_SQLite:/drv:ign-8.1.36-lab-primary:bip_tag_provider:/tag:' + tag
        pens.append({
            'name': tagName,
            'enabled': True,
            'visible': True,
            'selectable': True,
            'axis': '',
            'plot': 0,
            'data': {
                'source': dataSource,
                'aggregateMode': 'default'
            }
        })
    self.view.custom.pendingPens = pens

| Line | Code                                                                                          | What it does                                                                                                                                              |
| ---: | --------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def onMessageReceived(self, payload):`                                                       | Defines a Perspective message handler. Runs when this view receives a message, with `payload` as the message data (a dict-like object).                   |
|    2 | `tags = payload.get('tags', [])`                                                              | Reads `tags` from the payload. If the key is missing, defaults to an empty list so the rest of the script can run safely.                                 |
|    3 | `pens = []`                                                                                   | Creates an empty list that will hold pen configuration objects for a PowerChart.                                                                          |
|    4 | `for tag in tags:`                                                                            | Loops over each tag path string in the `tags` list.                                                                                                       |
|    5 | `tagName = tag.split('/')[-1]`                                                                | Splits the tag path by `/` and takes the last segment as a display name (the leaf tag name).                                                              |
|    6 | `dataSource = 'histprov:BiP_SQLite:/drv:ign-8.1.36-lab-primary:bip_tag_provider:/tag:' + tag` | Builds a historian datasource string for this pen by concatenating a fixed historian prefix with the full tag path.                                       |
|    7 | `pens.append({`                                                                               | Appends a new pen configuration dictionary to the `pens` list.                                                                                            |
|    8 | `'name': tagName,`                                                                            | Sets the pen name shown in the chart using the leaf tag name.                                                                                             |
|    9 | `'enabled': True,`                                                                            | Enables the pen so it participates in the chart configuration.                                                                                            |
|   10 | `'visible': True,`                                                                            | Makes the pen visible on the chart by default.                                                                                                            |
|   11 | `'selectable': True,`                                                                         | Allows the pen to be selected in the chart UI (legend interactions, selection features).                                                                  |
|   12 | `'axis': '',`                                                                                 | Leaves axis assignment blank, so the chart uses default axis behavior.                                                                                    |
|   13 | `'plot': 0,`                                                                                  | Assigns the pen to plot index 0 (the first plot).                                                                                                         |
|   14 | `'data': {`                                                                                   | Starts the nested configuration object that defines where the pen pulls its data from.                                                                    |
|   15 | `'source': dataSource,`                                                                       | Sets the pen’s data source to the historian path string built earlier.                                                                                    |
|   16 | `'aggregateMode': 'default'`                                                                  | Uses the default aggregation behavior for historian queries (chart determines aggregation based on range and settings).                                   |
|   17 | `}`                                                                                           | Ends the nested `data` dictionary.                                                                                                                        |
|   18 | `})`                                                                                          | Ends the pen dictionary and completes the append call.                                                                                                    |
|   19 | `self.view.custom.pendingPens = pens`                                                         | Stores the full pens list into a custom view property named `pendingPens`, making it available elsewhere in the view (bindings, scripts, button actions). |


| Item                           | Type       | Shape / Example                                                                                  |
| ------------------------------ | ---------- | ------------------------------------------------------------------------------------------------ |
| `payload`                      | dict-like  | `{'tags': ['Folder/Tag1', 'Folder/Tag2']}`                                                       |
| `tags`                         | list[str]  | Each element is a tag path string, using `/` separators                                          |
| `pens`                         | list[dict] | Each dict matches the structure expected by `PowerChart.props.pens`                              |
| `self.view.custom.pendingPens` | list[dict] | Holds the generated pen configs for later use (for example writing into `PowerChart.props.pens`) |


| Field                | Value in your code | Meaning                                   |
| -------------------- | ------------------ | ----------------------------------------- |
| `name`               | `tagName`          | Display name for the pen                  |
| `enabled`            | `True`             | Pen is active in configuration            |
| `visible`            | `True`             | Pen is drawn by default                   |
| `selectable`         | `True`             | User can select the pen in UI             |
| `axis`               | `''`               | Default axis selection behavior           |
| `plot`               | `0`                | Which plot this pen is assigned to        |
| `data.source`        | `dataSource`       | Historian datasource path the pen queries |
| `data.aggregateMode` | `'default'`        | Aggregation strategy used for the query   |


In [ ]:
#PowerChart - Property Change Script for pens
def valueChanged(self, previousValue, currentValue, origin, missedEvents):
	log = system.util.getLogger("PowerChartPens")

	prevPens = []
	currPens = []

	if previousValue is not None and previousValue.value is not None:
		prevPens = previousValue.value

	if currentValue is not None and currentValue.value is not None:
		currPens = currentValue.value

	prevCount = len(prevPens)
	currCount = len(currPens)

	if currCount > prevCount:
		log.info("pen added, auto stacking")

		# Rebuild plots equal to number of pens
		plots = []

		for i in range(currCount):
			plots.append({
				"relativeWeight": 1,
				"color": "#FFFFFF",
				"markers": [],
				"style": {"classes": ""}
			})

		self.props.plots = plots

		# Assign each pen to its own plot
		for i, pen in enumerate(self.props.pens):
			pen.plot = i

| Line | Code                                                                         | What it does                                                                                                   |
| ---: | ---------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- |
|    1 | `def valueChanged(self, previousValue, currentValue, origin, missedEvents):` | Defines a Perspective property change event script. Runs when the watched property changes value.              |
|    2 | `log = system.util.getLogger("PowerChartPens")`                              | Creates a named logger so messages from this script can be filtered in the Gateway logs.                       |
|    3 | `prevPens = []`                                                              | Initializes a safe default list for the previous pens value.                                                   |
|    4 | `currPens = []`                                                              | Initializes a safe default list for the current pens value.                                                    |
|    5 | `if previousValue is not None and previousValue.value is not None:`          | Confirms `previousValue` exists and contains a real value before reading it.                                   |
|    6 | `prevPens = previousValue.value`                                             | Copies the old pens array into `prevPens`.                                                                     |
|    7 | `if currentValue is not None and currentValue.value is not None:`            | Confirms `currentValue` exists and contains a real value before reading it.                                    |
|    8 | `currPens = currentValue.value`                                              | Copies the new pens array into `currPens`.                                                                     |
|    9 | `prevCount = len(prevPens)`                                                  | Counts how many pens were previously configured.                                                               |
|   10 | `currCount = len(currPens)`                                                  | Counts how many pens are currently configured.                                                                 |
|   11 | `if currCount > prevCount:`                                                  | Detects that a pen was added (new pens list is longer than the old one).                                       |
|   12 | `log.info("pen added, auto stacking")`                                       | Writes an informational log entry when a pen add is detected.                                                  |
|   13 | `# Rebuild plots equal to number of pens`                                    | Comment describing the next block: rebuild the plots list to match the pen count.                              |
|   14 | `plots = []`                                                                 | Initializes a list that will hold plot configuration dicts for the PowerChart.                                 |
|   15 | `for i in range(currCount):`                                                 | Loops once per pen, so it can create one plot per pen.                                                         |
|   16 | `plots.append({`                                                             | Appends one plot configuration dictionary into `plots`.                                                        |
|   17 | `"relativeWeight": 1,`                                                       | Sets plot height weighting so each plot gets equal space when stacked.                                         |
|   18 | `"color": "#FFFFFF",`                                                        | Sets the plot color setting for that plot configuration.                                                       |
|   19 | `"markers": [],`                                                             | Initializes an empty markers list for the plot.                                                                |
|   20 | `"style": {"classes": ""}`                                                   | Provides a style object for the plot, leaving CSS classes empty.                                               |
|   21 | `})`                                                                         | Closes the plot dictionary and finishes the append call.                                                       |
|   22 | `self.props.plots = plots`                                                   | Writes the newly built plots array into the PowerChart plots property, forcing plot layout to match pen count. |
|   23 | `# Assign each pen to its own plot`                                          | Comment describing the next block: map pen index to plot index one to one.                                     |
|   24 | `for i, pen in enumerate(self.props.pens):`                                  | Iterates through the chart’s pens with an index so each pen can be assigned to a unique plot.                  |
|   25 | `pen.plot = i`                                                               | Sets each pen’s `plot` property equal to its index, placing pen 0 in plot 0, pen 1 in plot 1, etc.             |


| Item                    | Before                     | After pen added                                 |
| ----------------------- | -------------------------- | ----------------------------------------------- |
| `currCount > prevCount` | False                      | True                                            |
| `self.props.plots`      | Existing plot list         | Rebuilt to length `currCount`                   |
| Pen plot assignment     | Whatever it was previously | Forced to one pen per plot (`pen.plot = index`) |


| Name                             | What it is               | Used for                                          |
| -------------------------------- | ------------------------ | ------------------------------------------------- |
| `system.util.getLogger(...)`     | Ignition logging utility | Writes to Gateway logs with a category name       |
| `previousValue` / `currentValue` | QualifiedValue objects   | Provide `.value` holding the underlying pens list |
| `self.props.pens`                | PowerChart pens array    | Enumerated to assign plot indices                 |
| `self.props.plots`               | PowerChart plots array   | Rebuilt to match pens for stacked layout          |


## History Popup

### Root

In [ ]:
#Event Configuration - onStartup
def runAction(self):
    self.view.custom.tableData = []
    self.view.custom.selectedTag = ""

| Line | Code                                | What it does                                                                                                                                |
| ---: | ----------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def runAction(self):`              | Defines a Perspective component action script. This function runs when the component’s configured action fires (commonly a button click).   |
|    2 | `self.view.custom.tableData = []`   | Clears the view custom property `tableData` by setting it to an empty list, which typically empties a table that is bound to this property. |
|    3 | `self.view.custom.selectedTag = ""` | Resets the view custom property `selectedTag` to an empty string, clearing any stored tag selection state.                                  |


| Expression         | Meaning                                        | Typical use                                     |
| ------------------ | ---------------------------------------------- | ----------------------------------------------- |
| `self`             | The component the action script is attached to | Provides access to the view and component tree  |
| `self.view`        | The view instance containing the component     | Used to access custom view-level state          |
| `self.view.custom` | Custom property container on the view          | Used to store UI state shared across components |
| `tableData`        | A custom property holding table rows           | Often bound to a Table component’s `props.data` |
| `selectedTag`      | A custom property holding a tag path or name   | Used to track what tag is currently selected    |


### Tag Browse Tree

In [ ]:
#Event Configuration - on TabBrowseTree - onNodeClick
def runAction(self, event):
	self.view.custom.selectedTag = self.props.selection.values[0]

| Line | Code                                                            | What it does                                                                                                                                                                                                |
| ---: | --------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def runAction(self, event):`                                   | Defines a Perspective action script that runs when the component’s configured action fires. `event` contains context about what triggered the action.                                                       |
|    2 | `self.view.custom.selectedTag = self.props.selection.values[0]` | Reads the first selected value from the component’s `props.selection.values` list and stores it into the view level custom property `selectedTag` so other components in the view can use the selected tag. |


| Part       | Expression                       | Meaning                                                                                                     |
| ---------- | -------------------------------- | ----------------------------------------------------------------------------------------------------------- |
| Left side  | `self.view.custom.selectedTag`   | The view-level custom property named `selectedTag` where you store the current selection.                   |
| Right side | `self.props.selection.values[0]` | The first element in the component’s selection values array, typically the selected tag path or identifier. |
| Operator   | `=`                              | Assigns the right side value into the left side property.                                                   |


| Item                           | Typical type | Notes                                                                       |
| ------------------------------ | ------------ | --------------------------------------------------------------------------- |
| `self`                         | Component    | The component the script is attached to (likely your Tag Browse Tree).      |
| `event`                        | Event object | Not used in this snippet, but available for additional context if needed.   |
| `self.props.selection.values`  | list         | Array of selected items. Index `[0]` assumes at least one selection exists. |
| `self.view.custom.selectedTag` | string       | Commonly used to store a tag path for other scripts or bindings.            |


### TagSelected (Table)

In [ ]:
# Custom Method to delete selected row from table and update data property
def delete_selected_row(self):
    if self.props.selection.selectedRow is not None:
        data = list(self.view.custom.tableData)
        data.pop(self.props.selection.selectedRow)
        self.view.custom.tableData = data
        self.props.data = data

| Line | Code                                               | What it does                                                                                                |
| ---: | -------------------------------------------------- | ----------------------------------------------------------------------------------------------------------- |
|    1 | `def delete_selected_row(self):`                   | Defines a custom function that deletes the currently selected row from a table’s data.                      |
|    2 | `if self.props.selection.selectedRow is not None:` | Checks that a row is actually selected. If nothing is selected, the function does nothing.                  |
|    3 | `data = list(self.view.custom.tableData)`          | Makes a new Python list copy of the view-level `tableData` so it can be modified safely.                    |
|    4 | `data.pop(self.props.selection.selectedRow)`       | Removes the element at the selected row index from the copied list.                                         |
|    5 | `self.view.custom.tableData = data`                | Writes the updated list back to the view-level `tableData` so other components and bindings see the change. |
|    6 | `self.props.data = data`                           | Updates this Table component’s `props.data` directly so the table UI reflects the updated data.             |


| Expression                                   | Part           | Meaning                                                                  |
| -------------------------------------------- | -------------- | ------------------------------------------------------------------------ |
| `self.props.selection.selectedRow`           | `self`         | The Table component this script runs on.                                 |
|                                              | `.props`       | The component’s properties container.                                    |
|                                              | `.selection`   | The Table’s selection state.                                             |
|                                              | `.selectedRow` | The integer index of the selected row, or `None` if nothing is selected. |
| `data.pop(self.props.selection.selectedRow)` | `data`         | The working list copy of your table rows.                                |
|                                              | `.pop(index)`  | Removes and returns the element at `index` from the list.                |
|                                              | `index`        | Uses the table’s selected row index.                                     |


| Step | Reads                              | Writes                       | Result                                |
| ---- | ---------------------------------- | ---------------------------- | ------------------------------------- |
| 1    | `self.props.selection.selectedRow` | none                         | Confirms a valid row is selected      |
| 2    | `self.view.custom.tableData`       | `data`                       | Copies current rows to a working list |
| 3    | `data`                             | `data`                       | Removes the selected row              |
| 4    | `data`                             | `self.view.custom.tableData` | Updates the view-level state          |
| 5    | `data`                             | `self.props.data`            | Updates the Table’s displayed data    |


In [ ]:
#Message Handler for delete button in table to remove selected row
def onMessageReceived(self, payload):
	self.delete_selected_row()

| Line | Code                                    | What it does                                                                                                                                                   |
| ---: | --------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def onMessageReceived(self, payload):` | Defines a Perspective message handler. It runs when this view or component receives a message. `payload` is the message data (usually a dict).                 |
|    2 | `self.delete_selected_row()`            | Calls a custom function named `delete_selected_row` on this same object to remove the currently selected table row. The `payload` is not used in this snippet. |


| Term                    | Meaning in this snippet                                                                                                |
| ----------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| `self`                  | The object the message handler is attached to (a component or the view, depending on where the handler is defined).    |
| `payload`               | The message contents sent by `system.perspective.sendMessage(...)` or similar, not used here.                          |
| `delete_selected_row()` | A user defined function that performs the row deletion logic. It must exist on the same `self` where this script runs. |


### Add Button

In [ ]:
# add button - onActionPerformed event configuration
def runAction(self, event):
    tagPath = self.view.custom.selectedTag
	
    if not tagPath:
        return

    data = list(self.view.custom.tableData) if self.view.custom.tableData else []

    if any(row.get("Tag") == tagPath for row in data):
        return

    data.append({"Tag": tagPath})

    self.view.custom.tableData = data
    self.parent.parent.getChild("TagSelected").props.data = data

| Line | Code                                                                            | What it does                                                                                                                             |
| ---: | ------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def runAction(self, event):`                                                   | Defines a Perspective action script that runs when the component action fires. `event` provides context but is not used here.            |
|    2 | `tagPath = self.view.custom.selectedTag`                                        | Reads the currently selected tag path from a view-level custom property and stores it in a local variable.                               |
|    3 | `if not tagPath:`                                                               | Checks if `tagPath` is empty or `None`.                                                                                                  |
|    4 | `return`                                                                        | Exits early if there is no selected tag to add.                                                                                          |
|    5 | `data = list(self.view.custom.tableData) if self.view.custom.tableData else []` | Creates a working list copy of `tableData` if it exists. If it is empty or missing, starts with a new empty list.                        |
|    6 | `if any(row.get("Tag") == tagPath for row in data):`                            | Scans the existing rows to see if any row already has `"Tag"` equal to the selected tag. This prevents duplicates.                       |
|    7 | `return`                                                                        | Exits early if the tag is already in the table.                                                                                          |
|    8 | `data.append({"Tag": tagPath})`                                                 | Appends a new row dictionary to the list with a single key `"Tag"` whose value is the selected tag path.                                 |
|    9 | `self.view.custom.tableData = data`                                             | Writes the updated list back to the view-level custom property so other components and bindings can see it.                              |
|   10 | `self.parent.parent.getChild("TagSelected").props.data = data`                  | Navigates up two container levels, finds the child component named `TagSelected`, and sets its table data so the UI updates immediately. |


| Expression                                   | Meaning                                                                                 |
| -------------------------------------------- | --------------------------------------------------------------------------------------- |
| `self.view.custom.selectedTag`               | Shared view-level variable containing the current tag selection.                        |
| `self.view.custom.tableData`                 | Shared view-level list of table row dictionaries.                                       |
| `list(...)`                                  | Makes a new list so you can modify it without relying on the original object reference. |
| `any(...)`                                   | Returns `True` if at least one row matches, otherwise `False`.                          |
| `row.get("Tag")`                             | Reads `"Tag"` from the row dict, returning `None` if the key does not exist.            |
| `self.parent.parent.getChild("TagSelected")` | Component tree navigation to reach the table component by name.                         |


### Remove Button

In [ ]:
def runAction(self, event):
	system.perspective.sendMessage('delete_selected_row')

| Line | Code                                                    | What it does                                                                                                                                                                  |
| ---: | ------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def runAction(self, event):`                           | Defines a Perspective action script that runs when the component’s action fires. `event` contains context about the action but is not used here.                              |
|    2 | `system.perspective.sendMessage('delete_selected_row')` | Sends a Perspective message with the message type `'delete_selected_row'`. Any configured message handler listening for this message type (within the target scope) will run. |


| What happens next     | Description                                                                                                                                            |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------ |
| Message handlers fire | Any component or view that has an `onMessageReceived` handler configured for the message type `'delete_selected_row'` will execute its handler script. |
| `payload`             | If you do not provide a payload argument, handlers typically receive an empty payload.                                                                 |
| Scope                 | Without specifying scope parameters, it uses Perspective’s default behavior for where the message is delivered, based on the function call context.    |


### Add All

In [ ]:
def runAction(self, event):
    tags = [row['Tag'] for row in self.view.custom.tableData]
    validTags = []
    for tag in tags:
        try:
            result = system.tag.queryTagHistory(
                paths=['[BiP_Tag_Provider]' + tag],
                startDate=system.date.addMinutes(system.date.now(), -1),
                endDate=system.date.now()
            )
            if result.getRowCount() > 0:
                validTags.append(tag)
        except:
            pass
    system.perspective.sendMessage('plotTags', payload={'tags': validTags}, scope='session')
    system.perspective.closePopup('TH31T6oq')

| Line | Code                                                                                       | What it does                                                                                                                                                                |
| ---: | ------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
|    1 | `def runAction(self, event):`                                                              | Defines a Perspective action script. Runs when the component action fires.                                                                                                  |
|    2 | `tags = [row['Tag'] for row in self.view.custom.tableData]`                                | Builds a list of tag path strings by pulling the `"Tag"` value from each row dict in `view.custom.tableData`.                                                               |
|    3 | `validTags = []`                                                                           | Initializes an empty list to hold only the tags that pass the history query check.                                                                                          |
|    4 | `for tag in tags:`                                                                         | Loops through each tag path string.                                                                                                                                         |
|    5 | `try:`                                                                                     | Starts an error handling block so a bad tag does not crash the whole script.                                                                                                |
|    6 | `result = system.tag.queryTagHistory(`                                                     | Runs a historian query for the current tag. The returned object is a dataset of history samples.                                                                            |
|    7 | `paths=['[BiP_Tag_Provider]' + tag],`                                                      | Builds a full tag path by prefixing the provider name, and passes it as a one element list to the query.                                                                    |
|    8 | `startDate=system.date.addMinutes(system.date.now(), -1),`                                 | Sets the query start time to 1 minute ago from now.                                                                                                                         |
|    9 | `endDate=system.date.now()`                                                                | Sets the query end time to now.                                                                                                                                             |
|   10 | `)`                                                                                        | Ends the `queryTagHistory` call.                                                                                                                                            |
|   11 | `if result.getRowCount() > 0:`                                                             | Checks if the dataset contains at least one row of history data.                                                                                                            |
|   12 | `validTags.append(tag)`                                                                    | Adds the tag to `validTags` if it returned history data.                                                                                                                    |
|   13 | `except:`                                                                                  | Catches any exception thrown by the query or path building.                                                                                                                 |
|   14 | `pass`                                                                                     | Ignores the error and continues to the next tag in the loop.                                                                                                                |
|   15 | `system.perspective.sendMessage('plotTags', payload={'tags': validTags}, scope='session')` | Sends a Perspective message of type `'plotTags'` with a payload containing the filtered tag list, delivered to the session scope so listeners in the session can handle it. |
|   16 | `system.perspective.closePopup('TH31T6oq')`                                                | Closes the Perspective popup with the id `'TH31T6oq'` after sending the message.                                                                                            |


| Expression                        | Meaning                | Notes                                                                                    |
| --------------------------------- | ---------------------- | ---------------------------------------------------------------------------------------- |
| `[row['Tag'] for row in ...]`     | List comprehension     | Extracts the `"Tag"` field from each row dict.                                           |
| `system.tag.queryTagHistory(...)` | Historian query        | Returns a dataset of samples in the time range.                                          |
| `result.getRowCount()`            | Dataset row count      | Used as a quick “did I get data” check.                                                  |
| `try/except`                      | Error handling         | Prevents one bad tag from stopping the whole action.                                     |
| `scope='session'`                 | Message delivery scope | Ensures the message is delivered to handlers within the session, not only within a view. |
| `payload={'tags': validTags}`     | Message data           | Makes the tag list available as `payload['tags']` in the message handler.                |
